In [ ]:
import logging
import os
import time
import lotr

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s: %(message)s')

In [ ]:
conf = lotr.read_conf()

In [ ]:
timestamp = time.time()
lotr.download_sheet(conf)
lotr.extract_data(conf)
sets = lotr.get_sets(conf)
lotr.sanity_check(conf, sets)
lotr.save_data_for_bot(conf)

if conf['output_languages']:
    lotr.reset_project_folders(conf)

strange_eons = False
changes = False
for set_id, set_name in sets:
    if conf['octgn_set_xml']:
        lotr.generate_octgn_set_xml(conf, set_id, set_name)

    if conf['octgn_o8d']:
        lotr.generate_octgn_o8d(conf, set_id, set_name)

    if conf['ringsdb_csv']:
        lotr.generate_ringsdb_csv(conf, set_id, set_name)

    if conf['dragncards_json']:
        lotr.generate_dragncards_json(conf, set_id, set_name)

    if conf['hallofbeorn_json']:
        lotr.generate_hallofbeorn_json(conf, set_id, set_name)

    if conf['frenchdb_csv']:
        lotr.generate_frenchdb_csv(conf, set_id, set_name)

    if conf['spanishdb_csv']:
        lotr.generate_spanishdb_csv(conf, set_id, set_name)

    if conf['output_languages']:
        lotr.copy_custom_images(conf, set_id, set_name)

    for lang in conf['output_languages']:
        strange_eons = True
        lotr.generate_xml(conf, set_id, set_name, lang)
        lotr.update_xml(conf, set_id, set_name, lang)
        new_hash, old_hash = lotr.calculate_hashes(set_id, set_name, lang)
        if new_hash != old_hash:
            changes = True

        lotr.copy_raw_images(conf, set_id, set_name, lang)
        lotr.copy_xml(set_id, set_name, lang)

if conf['octgn_set_xml'] or conf['octgn_o8d']:
    lotr.copy_octgn_outputs(conf, sets)

if conf['ringsdb_csv'] and conf['update_ringsdb']:
    lotr.update_ringsdb(conf, sets)

if changes:
    lotr.create_project()
elif strange_eons:
    logging.info('No changes since the last run, skipping creating '
                 'Strange Eons project')
else:
    logging.info('No Strange Eons outputs, skipping creating Strange Eons '
                 'project')

logging.info('Done (%ss)', round(time.time() - timestamp, 3))

### Strange Eons Part:

1. Open `setGenerator.seproject` in Strange Eons.
2. Run `Script/makeCards` script by double clicking it.
3. Once completed, close Strange Eons (wait until it finished packing the project).

In [ ]:
timestamp = time.time()
for set_id, set_name in sets:
    for lang in conf['output_languages']:
        skip_set, skip_ids = lotr.get_skip_info(set_id, set_name, lang)
        if skip_set:
            logging.info('[%s, %s] No changes since the last run,'
                         ' skipping', set_name, lang)
            continue

        if conf['nobleed_300'][lang]:
            lotr.generate_png300_nobleed(conf, set_id, set_name, lang, skip_ids)

        if conf['nobleed_800'][lang]:
            lotr.generate_png800_nobleed(conf, set_id, set_name, lang, skip_ids)

        if 'db' in conf['outputs'][lang]:
            lotr.generate_png300_db(conf, set_id, set_name, lang, skip_ids)
            lotr.generate_db(conf, set_id, set_name, lang, lotr.translated_data(set_id, lang))

        if 'octgn' in conf['outputs'][lang]:
            lotr.generate_png300_octgn(set_id, set_name, lang, skip_ids)
            lotr.generate_octgn(conf, set_id, set_name, lang)

        if 'rules_pdf' in conf['outputs'][lang]:
            lotr.generate_png300_rules_pdf(set_id, set_name, lang, skip_ids, lotr.translated_data(set_id, lang))
            lotr.generate_rules_pdf(conf, set_id, set_name, lang)

        if 'pdf' in conf['outputs'][lang]:
            lotr.generate_png300_pdf(conf, set_id, set_name, lang, skip_ids)
            lotr.generate_pdf(conf, set_id, set_name, lang)

        if 'genericpng_pdf' in conf['outputs'][lang]:
            lotr.generate_png800_pdf(conf, set_id, set_name, lang, skip_ids)
            lotr.generate_genericpng_pdf(conf, set_id, set_name, lang)

        if 'makeplayingcards' in conf['outputs'][lang]:
            lotr.generate_png800_bleedmpc(conf, set_id, set_name, lang, skip_ids)
            lotr.generate_mpc(conf, set_id, set_name, lang, lotr.translated_data(set_id, lang))

        if 'drivethrucards' in conf['outputs'][lang]:
            lotr.generate_jpg300_bleeddtc(conf, set_id, set_name, lang, skip_ids)
            lotr.generate_dtc(conf, set_id, set_name, lang, lotr.translated_data(set_id, lang))

        if 'mbprint' in conf['outputs'][lang]:
            lotr.generate_jpg800_bleedmbprint(conf, set_id, set_name, lang, skip_ids)
            lotr.generate_mbprint(conf, set_id, set_name, lang, lotr.translated_data(set_id, lang))

        if 'genericpng' in conf['outputs'][lang]:
            lotr.generate_png800_bleedgeneric(conf, set_id, set_name, lang, skip_ids)
            lotr.generate_genericpng(conf, set_id, set_name, lang, lotr.translated_data(set_id, lang))

if conf['db_destination_path'] and 'db' in conf['outputs'].get('English', {}):
    lotr.copy_db_outputs(conf, sets)

if conf['octgn_image_destination_path'] and 'octgn' in conf['outputs'].get('English', {}):
    lotr.copy_octgn_image_outputs(conf, sets)

if conf['upload_dragncards'] and conf['dragncards_hostname'] and conf['dragncards_id_rsa_path']:
    lotr.upload_dragncards(conf, sets)

if conf['remote_logs_path']:
    with open(os.path.join(conf['remote_logs_path'], 'last_generated_image'), 'w') as fobj:
        fobj.write(lotr.get_last_image_timestamp())

logging.info('Done (%ss)', round(time.time() - timestamp, 3))